# T1140 - Deobfuscate/Decode Files or Information
Adversaries may use [Obfuscated Files or Information](https://attack.mitre.org/techniques/T1027) to hide artifacts of an intrusion from analysis. They may require separate mechanisms to decode or deobfuscate that information depending on how they intend to use it. Methods for doing that include built-in functionality of malware or by using utilities present on the system.

One such example is use of [certutil](https://attack.mitre.org/software/S0160) to decode a remote access tool portable executable file that has been hidden inside a certificate file. (Citation: Malwarebytes Targeted Attack against Saudi Arabia) Another example is using the Windows <code>copy /b</code> command to reassemble binary fragments into a malicious payload. (Citation: Carbon Black Obfuscation Sept 2016)

Sometimes a user's action may be required to open it for deobfuscation or decryption as part of [User Execution](https://attack.mitre.org/techniques/T1204). The user may also be required to input a password to open a password protected compressed/encrypted file that was provided by the adversary. (Citation: Volexity PowerDuke November 2016)

## Atomic Tests

### Atomic Test #1 - Deobfuscate/Decode Files Or Information
Encode/Decode executable
Upon execution a file named T1140_calc_decoded.exe will be placed in the temp folder

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
certutil -encode C:\Windows\System32\calc.exe %temp%\T1140_calc.txt
certutil -decode %temp%\T1140_calc.txt %temp%\T1140_calc_decoded.exe
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 1

### Atomic Test #2 - Certutil Rename and Decode
Rename certutil and decode a file. This is in reference to latest research by FireEye [here](https://www.fireeye.com/blog/threat-research/2018/09/apt10-targeting-japanese-corporations-using-updated-ttps.html)

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %windir%\system32\certutil.exe %temp%\tcm.tmp
%temp%\tcm.tmp -encode C:\Windows\System32\calc.exe %temp%\T1140_calc2.txt
%temp%\tcm.tmp -decode %temp%\T1140_calc2.txt %temp%\T1140_calc2_decoded.exe
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 2

### Atomic Test #3 - Base64 decoding with Python
Use Python to decode a base64-encoded text string and echo it to the console

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `None`!
##### Description: Python must be present

##### Check Prereq Commands:
```None
which python3

```
##### Get Prereq Commands:
```None
echo "Please install Python 3"

```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ENCODED=$(python3 -c 'import base64;enc=base64.b64encode("Hello from Atomic Red Team test T1140!".encode());print(enc.decode())')
python3 -c "import base64;dec=base64.b64decode(\"$ENCODED\");print(dec.decode())"
python3 -c "import base64 as d;dec=d.b64decode(\"$ENCODED\");print(dec.decode())"
python3 -c "from base64 import b64decode;dec=b64decode(\"$ENCODED\");print(dec.decode())"
python3 -c "from base64 import b64decode as d;dec=d(\"$ENCODED\");print(dec.decode())"
echo $ENCODED | python3 -c "import base64,sys;dec=base64.b64decode(sys.stdin.read());print(dec.decode())"
echo $ENCODED > /tmp/T1140.encoded && python3 -c "import base64;dec=base64.b64decode(open('/tmp/T1140.encoded').read());print(dec.decode())"
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 3

### Atomic Test #4 - Base64 decoding with Perl
Use Perl to decode a base64-encoded text string and echo it to the console

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `None`!
##### Description: Perl must be present

##### Check Prereq Commands:
```None
which perl

```
##### Get Prereq Commands:
```None
echo "Please install Perl"

```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ENCODED=$(perl -e "use MIME::Base64;print(encode_base64('Hello from Atomic Red Team test T1140!'));")
perl -le "use MIME::Base64;print(decode_base64('$ENCODED'));"
echo $ENCODED | perl -le 'use MIME::Base64;print(decode_base64(<STDIN>));'
echo $ENCODED > /tmp/T1140.encoded && perl -le 'use MIME::Base64;open($f,"<","/tmp/T1140.encoded");print(decode_base64(<$f>));'
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 4

### Atomic Test #5 - Base64 decoding with shell utilities
Use common shell utilities to decode a base64-encoded text string and echo it to the console

**Supported Platforms:** linux, macos
#### Attack Commands: Run with `sh`
```sh
ENCODED=$(echo 'Hello from Atomic Red Team test T1140!' | base64)
printf $ENCODED | base64 -d
echo $ENCODED | base64 -d
echo $(echo $ENCODED) | base64 -d
echo $ENCODED > /tmp/T1140.encoded && base64 -d /tmp/T1140.encoded
echo $ENCODED > /tmp/T1140.encoded && base64 -d < /tmp/T1140.encoded
echo $ENCODED > /tmp/T1140.encoded && cat /tmp/T1140.encoded | base64 -d
echo $ENCODED > /tmp/T1140.encoded && cat < /tmp/T1140.encoded | base64 -d
bash -c "{echo,\"$(echo $ENCODED)\"}|{base64,-d}"
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 5

### Atomic Test #6 - Hex decoding with shell utilities
Use common shell utilities to decode a hex-encoded text string and echo it to the console

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `None`!
##### Description: xxd must be present

##### Check Prereq Commands:
```None
which xxd

```
##### Get Prereq Commands:
```None
echo "Please install xxd"

```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ENCODED=$(echo 'Hello from Atomic Red Team test T1140!' | xxd -ps -c 256)
printf $ENCODED | xxd -r -p
echo $ENCODED | xxd -r -p
echo $(echo $ENCODED) | xxd -r -p
echo $ENCODED > /tmp/T1140.encoded && xxd -r -p /tmp/T1140.encoded
echo $ENCODED > /tmp/T1140.encoded && xxd -r -p < /tmp/T1140.encoded
echo $ENCODED > /tmp/T1140.encoded && cat /tmp/T1140.encoded | xxd -r -p
echo $ENCODED > /tmp/T1140.encoded && cat < /tmp/T1140.encoded | xxd -r -p
```

In [ ]:
Invoke-AtomicTest T1140 -TestNumbers 6

## Detection
Detecting the action of deobfuscating or decoding files or information may be difficult depending on the implementation. If the functionality is contained within malware and uses the Windows API, then attempting to detect malicious behavior before or after the action may yield better results than attempting to perform analysis on loaded libraries or API calls. If scripts are used, then collecting the scripts for analysis may be necessary. Perform process and command-line monitoring to detect potentially malicious behavior related to scripts and system utilities such as [certutil](https://attack.mitre.org/software/S0160).

Monitor the execution file paths and command-line arguments for common archive file applications and extensions, such as those for Zip and RAR archive tools, and correlate with other suspicious behavior to reduce false positives from normal user and administrator behavior.

## Shield Active Defense
### API Monitoring 
 Monitor local APIs that might be used by adversary tools and activity. 

 API Monitoring involves capturing an internal Operating System (OS) function for its usage, accompanying arguments, and result. When a defender captures this information, the intelligence gathered can be analyzed to gain insight into the activity of an adversary at a level deeper than normal system activity monitoring.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can monitor and analyze operating system functions calls for detection and alerting.
#### Procedures
Trace activity through WinSock TCP API functions to view potentially  malicious network events. Log it such that it can be pushed to a centralized location and analyzed further.
Hook the Win32 DeleteFile() function to log all attempts at deleting a given file. This information can be used to trigger restoration attempts on critical data, reducing potential disruption if those files are unavailable for prolonged periods of time.